In [2]:
import gymnasium as gym

import time
import os

%cd ../..
%pip install --use-pep517 -e .
%load_ext autoreload
%autoreload 2

C:\Users\Hao\Downloads\RL
Obtaining file:///C:/Users/Hao/Downloads/RL
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for BC (pyproject.toml): started
  Building editable for BC (pyproject.toml): finished with status 'done'
  Created wheel for BC: filename=BC-0.1.0-0.editable-py3-none-any.whl size=2542 sha256=d81c0440c66b34958f3b25f708c1f85d6cbb282397f2d96ec6db8c82a8354ca0
  Stored in directory: C:\Users\Hao\AppData\Local\Temp\pip-ephem-wheel-cache-6q_7twsf\wheels\ce\10\5b\82fb1cd28b0c9ba2dc72e445e4dcdf18c9e07949664bd0c133
Suc

In [3]:
from BC.infrastructure.notebook_util import *
from BC.scripts.bc_dagger_runner import run_training_loop

In [4]:
env = gym.make("CartPole-v1", render_mode="rgb_array")  # Needed for rendering
env = wrap_env(env)

env.reset()
for _ in range(100):
    obs, reward, terminated, truncated, info = env.step(env.action_space.sample())  # Take random actions
    if terminated or truncated:
        obs, _ = env.reset()

env.close()

print('Loading video...')
show_video(env)

C:\Users\Hao\anaconda3\envs\RL\lib\site-packages\gymnasium\wrappers\rendering.py:283: UserWarning: WARN: Overwriting existing videos at C:\Users\Hao\Downloads\RL\video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
C:\Users\Hao\anaconda3\envs\RL\lib\site-packages\pygame\pkgdata.py:25: UserWarning: Module BC was already imported from None, but c:\users\hao\downloads\rl is being added to sys.path
  from pkg_resources import resource_stream, resource_exists


Loading video...
video/CartPole-v1*.mp4


In [5]:
env = gym.make("Ant-v5", render_mode="rgb_array")  # Needed for rendering
env = wrap_env(env)

env.reset()
for _ in range(100):
    obs, reward, terminated, truncated, info = env.step(env.action_space.sample())  # Take random actions
    if terminated or truncated:
        obs, _ = env.reset()

env.close()

print('Loading video...')
show_video(env)

C:\Users\Hao\anaconda3\envs\RL\lib\site-packages\gymnasium\wrappers\rendering.py:283: UserWarning: WARN: Overwriting existing videos at C:\Users\Hao\Downloads\RL\video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Loading video...
video/Ant-v5*.mp4


In [6]:
env = gym.make("HalfCheetah-v5", render_mode="rgb_array")  # Needed for rendering
env = wrap_env(env)

env.reset()
for _ in range(100):
    obs, reward, terminated, truncated, info = env.step(env.action_space.sample())  # Take random actions
    if terminated or truncated:
        obs, _ = env.reset()

env.close()

print('Loading video...')
show_video(env)

Loading video...
video/HalfCheetah-v5*.mp4


## Behavior Cloning

In [7]:
class Args:

  def __getitem__(self, key):
    return getattr(self, key)

  def __setitem__(self, key, val):
    setattr(self, key, val)

  # expert data
  expert_policy_file:str = 'BC/policies/experts/HalfCheetah.pkl' 
  expert_data:str = 'BC/expert_data/expert_data_HalfCheetah-v4.pkl' 
  env_name:str = 'HalfCheetah-v4' # ['Ant-v4', 'Walker2d-v4', 'HalfCheetah-v4', 'Hopper-v4']
  exp_name:str = 'bc_halfCheetah' 
  do_dagger:bool = False
  ep_len:int = 1000
  save_params = False

  num_agent_train_steps_per_iter:int = 1000
  n_iter:int = 1

  # batches & buffers
  batch_size_initial:int = 2000 
  batch_size:int = 1000 
  eval_batch_size:int = 1000
  train_batch_size:int = 100
  max_replay_buffer_size:int = 1000_000

  # MLP
  n_layers:int = 2
  size:int = 64
  learning_rate:float = 5e-3
    
  # logging
  video_log_freq:int = 5 
  scalar_log_freq:int = 1 

  # gpu & run-time settings
  no_gpu:bool = False
  which_gpu:int = 0 
  seed:int = 42

args = Args()

params = vars(args)

if args.do_dagger:
    logdir_prefix = 'dagger_'
    assert args.n_iter > 1, "DAgger requires multiple iterations"  
else:
    logdir_prefix = 'behavior_cloning_'
    assert args.n_iter == 1, "Without DAgger, only one iteration is allowed"

data_path = 'data'
if not os.path.exists(data_path):
    os.makedirs(data_path)
logdir = logdir_prefix + args.exp_name + '_' + args.env_name + '_' + time.strftime("%d-%m-%Y_%H-%M-%S")
logdir = os.path.join(data_path, logdir)
params['logdir'] = logdir
if not os.path.exists(logdir):
    os.makedirs(logdir)

In [8]:
print(args.logdir)

data\behavior_cloning_bc_halfCheetah_HalfCheetah-v4_02-02-2025_10-10-37


In [9]:
run_training_loop(args)

###########################
logging outputs to  data\behavior_cloning_bc_halfCheetah_HalfCheetah-v4_02-02-2025_10-10-37
###########################
True
Using GPU 0


C:\Users\Hao\anaconda3\envs\RL\lib\site-packages\gymnasium\envs\registration.py:517: DeprecationWarning: WARN: The environment HalfCheetah-v4 is out of date. You should consider upgrading to version `v5`.
  logger.deprecation(


Loading expert policy from... BC/policies/experts/HalfCheetah.pkl
obs (1, 17) (1, 17)
Expert policy loaded.


********** Iteration 0 ************


(1000, 3, 250, 250)
(1000, 3, 250, 250)

	Eval_AverageReturn: -730.72607421875
	Eval_StdReturn: 0.0
	Eval_MaxReturn: -730.72607421875
	Eval_MinReturn: -730.72607421875
	Eval_AverageEpLen: 1000.0
	Train_AverageReturn: 4034.7999834965067
	Train_StdReturn: 32.8677631311341
	Train_MaxReturn: 4067.6677466276406
	Train_MinReturn: 4001.9322203653724
	Train_AverageEpLen: 1000.0
	Train_EnvstepsSoFar: 0
	TimeSinceStart: 23.35699486732483
Done logging...



Training agent...

Logging training statistics...

(1000, 3, 250, 250)
(1000, 3, 250, 250)

	Eval_AverageReturn: 4095.179931640625
	Eval_StdReturn: 0.0
	Eval_MaxReturn: 4095.179931640625
	Eval_MinReturn: 4095.179931640625
	Eval_AverageEpLen: 1000.0
	Train_AverageReturn: 4034.7999834965067
	Train_StdReturn: 32.8677631311341
	Train_MaxReturn: 4067.6677466276406
	Train_MinReturn: 4001.9322203653724
	T

In [11]:
class Args:

  def __getitem__(self, key):
    return getattr(self, key)

  def __setitem__(self, key, val):
    setattr(self, key, val)

  # expert data
  expert_policy_file:str = 'BC/policies/experts/HalfCheetah.pkl' 
  expert_data:str = 'BC/expert_data/expert_data_HalfCheetah-v4.pkl' 
  env_name:str = 'HalfCheetah-v4' # ['Ant-v4', 'Walker2d-v4', 'HalfCheetah-v4', 'Hopper-v4']
  exp_name:str = 'dagger_halfCheetah' 
  do_dagger:bool = True
  ep_len:int = 1000
  save_params = False

  num_agent_train_steps_per_iter:int = 1000
  n_iter:int = 5

  # batches & buffers
  batch_size_initial:int = 2000 
  batch_size:int = 1000 
  eval_batch_size:int = 1000
  train_batch_size:int = 100
  max_replay_buffer_size:int = 1000_000

  # MLP
  n_layers:int = 2
  size:int = 64
  learning_rate:float = 5e-3
    
  # logging
  video_log_freq:int = 5 
  scalar_log_freq:int = 1 

  # gpu & run-time settings
  no_gpu:bool = False
  which_gpu:int = 0 
  seed:int = 42

args = Args()

params = vars(args)

if args.do_dagger:
    logdir_prefix = 'dagger_'
    assert args.n_iter > 1, "DAgger requires multiple iterations"  
else:
    logdir_prefix = 'behavior_cloning_'
    assert args.n_iter == 1, "Without DAgger, only one iteration is allowed"

data_path = 'data'
if not os.path.exists(data_path):
    os.makedirs(data_path)
logdir = logdir_prefix + args.exp_name + '_' + args.env_name + '_' + time.strftime("%d-%m-%Y_%H-%M-%S")
logdir = os.path.join(data_path, logdir)
params['logdir'] = logdir
if not os.path.exists(logdir):
    os.makedirs(logdir)

In [12]:
run_training_loop(args)

###########################
logging outputs to  data\dagger_dagger_halfCheetah_HalfCheetah-v4_02-02-2025_10-15-11
###########################
True
Using GPU 0
Loading expert policy from... BC/policies/experts/HalfCheetah.pkl
obs (1, 17) (1, 17)
Expert policy loaded.


********** Iteration 0 ************


(1000, 3, 250, 250)
(1000, 3, 250, 250)

	Eval_AverageReturn: -730.72607421875
	Eval_StdReturn: 0.0
	Eval_MaxReturn: -730.72607421875
	Eval_MinReturn: -730.72607421875
	Eval_AverageEpLen: 1000.0
	Train_AverageReturn: 4034.7999834965067
	Train_StdReturn: 32.8677631311341
	Train_MaxReturn: 4067.6677466276406
	Train_MinReturn: 4001.9322203653724
	Train_AverageEpLen: 1000.0
	Train_EnvstepsSoFar: 0
	TimeSinceStart: 23.069610595703125
Done logging...



Training agent...

Logging training statistics...

(1000, 3, 250, 250)
(1000, 3, 250, 250)

	Eval_AverageReturn: 4095.179931640625
	Eval_StdReturn: 0.0
	Eval_MaxReturn: 4095.179931640625
	Eval_MinReturn: 4095.179931640625
	Eval_AverageEpLen:

In [13]:
%load_ext tensorboard
%tensorboard --logdir ./data

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 19156), started 0:17:45 ago. (Use '!kill 19156' to kill it.)